##### install pymongo
- mac
    - pip(3) install pymongo
- windows
    - conda install -c anaconda pymongo

In [2]:
import pymongo, requests
import pandas as pd

In [3]:
# client 연결
ip = "13.124.198.120"
client = pymongo.MongoClient("mongodb://{ip}:27017".format(ip=ip))
client

MongoClient(host=['13.124.198.120:27017'], document_class=dict, tz_aware=False, connect=True)

In [4]:
# dss 데이터 베이스 선택
db = client.dss
db

Database(MongoClient(host=['13.124.198.120:27017'], document_class=dict, tz_aware=False, connect=True), 'dss')

In [5]:
# 데이터 베이스의 컬렉션 리스트 확인
db.collection_names()

['user', 'info1', 'info', 'articles']

In [6]:
# info 컬렉션 선택
collection = db.info
collection

Collection(Database(MongoClient(host=['13.124.198.120:27017'], document_class=dict, tz_aware=False, connect=True), 'dss'), 'info')

##### find

In [8]:
# find_one - 한개의 도큐먼트 가져오기
document = collection.find_one({"subject":"sass"})
type(document), document

(dict,
 {'_id': ObjectId('5b591f391a823899fc66b040'),
  'comments': [{'msg': 'hello', 'name': 'alice'}],
  'level': 4.0,
  'subject': 'sass'})

In [19]:
# find - 여러개의 도큐먼트를 가져오기
documents = collection.find()
documents

In [20]:
datas = list(documents)
len(datas)

8

In [21]:
list(documents)

[]

In [23]:
# count
documents = collection.find()
documents.count()

8

In [26]:
# query 문을 추가해서 find, sort
# 레벨이 3이상인 도큐먼트를 내림차순으로 정렬
# DESCENDING : -1, ASCENDING : 1
documents = collection.find( {"level": {"$lte":3}} ).sort("level", pymongo.DESCENDING)
list(documents)

[{'_id': ObjectId('5b591f391a823899fc66b03d'),
  'comments': [{'msg': 'check', 'name': 'po'}],
  'level': 3.0,
  'subject': 'java'},
 {'_id': ObjectId('5b591f391a823899fc66b03e'),
  'comments': [{'msg': 'nice', 'name': 'jin'}],
  'level': 3.0,
  'subject': 'javascript'},
 {'_id': ObjectId('5b591f391a823899fc66b03c'),
  'comments': [{'msg': 'bad', 'name': 'jin'}, {'msg': 'hi', 'name': 'alice'}],
  'level': 1.0,
  'subject': 'r'},
 {'_id': ObjectId('5b591f391a823899fc66b03f'),
  'comments': [],
  'level': 1.0,
  'subject': 'html'},
 {'_id': ObjectId('5b59220f28ae716578113a07'),
  'comments': [{'msg': 'hello', 'name': 'alice'}],
  'level': 1.0,
  'subject': 'less'}]

In [28]:
pymongo.DESCENDING, pymongo.ASCENDING

(-1, 1)

##### insert

In [29]:
# 한개의 데이터 저장
data = {
    "subject":"css", 
    "level":1, 
    "comments": [ {"name":"peter", "msg":"easy"} ]
}
result = collection.insert_one(data)
result

In [30]:
result.inserted_id

ObjectId('5b592eb946fb2a4d5313a052')

In [31]:
# 여러개의 데이터 저장
datas = [
    { "subject":"webpack", "level":4, "comments": [ {"name":"peter", "msg":"easy"} ] },
    { "subject":"html", "level":1, "comments": [ {"name":"peter", "msg":"easy"} ] },
    { "subject":"gulp", "level":2, "comments": [ {"name":"peter", "msg":"easy"} ] },
]
result = collection.insert_many(datas)
result

In [32]:
result.inserted_ids

[ObjectId('5b592f8646fb2a4d5313a053'),
 ObjectId('5b592f8646fb2a4d5313a054'),
 ObjectId('5b592f8646fb2a4d5313a055')]

##### 직방 API를 이용하여 데이터를 크롤링한 후에 바로 mongodb에 저장
- 직방 api url를 알아내기
- requests를 이용하여 api url로 json 포멧의 데이터를 가져오기
- json 데이터 포멧을 우리가 원하는 데이터 형태로 파싱 -> 리스트(딕셔너리1,딕셔너리2...)
- insert_many를 이용하여 mongodb에 데이터 저장

In [49]:
url = "https://api.zigbang.com/v3/items?detail=true&item_ids=[11845934,11759101,12146554,12247700,12170322,12181511,12185946,12195176,12016750,12129123,12236544,12075243,12207509,12206185,12257458,12133011,12252976,12226634,12268716,12150608,12103695,12195196,12240584,12236559,12217207,12264847,12257484,12068922,12075266,12182904,12262511,12080274,12225719,12252795,12197909,12107681,12117328,12247394,11998679,12154891,12112034,12255857,11620927,12116471,12236526,12182871,12254303,12036696,11766014,12225326,12206956,12088339,12241830,12169127,12206845,12209390,12250269,12199973,11852255,12156741]"
response = requests.get(url)
response

<Response [200]>

In [50]:
zigbang_dict_list = response.json().get("items")
len(zigbang_dict_list)

60

In [35]:
zigbang_dict_list[0]

{'header': False,
 'header_height': 0,
 'item': {'_floor': '고층/15층',
  'address1': '서울시 마포구 성산동',
  'address2': '593-3',
  'address3': None,
  'agent_address1': '서울특별시 마포구 모래내로1길 8 나동？다동 103호 (성산동)',
  'agent_comment': '\n━━━━━━━━━━━━━━━━━━━━━━━\n\n월드컵삼영공인중개사사무소 ☎02-309-3030\n\n친절하고 신속.정확한 중개 약속드립니다!\n많은 물건이 있으니 편하게 전화주세요~ \n\n오늘도 역시 밝게 웃을 수 있는 하루되시길 바랍니다^^ ',
  'agent_email': '3030land@naver.com',
  'agent_lat': 37.56481051112199,
  'agent_lng': 126.904238840368,
  'agent_local1': '서울시',
  'agent_local2': '마포구',
  'agent_mobile': '0507-1282-2194',
  'agent_name': '월드컵삼영공인중개사(서귀선)',
  'agent_no': 0,
  'agent_phone': '02-309-3030',
  'bjd_code': '1144012500',
  'bonbun_code': '593',
  'bubun_code': '3',
  'building': {'address': '서울시 마포구 성산동 593-3',
   'address2': '593-3',
   'building_id': 20073,
   'count': 107,
   'elevator': '1',
   'established': '2018.06',
   'floor': '15층',
   'lat': 37.56548690795898,
   'lng': 126.90431213378906,
   'local1': '서울시',
   'local2': '마포구',
   'loca

In [51]:
items = [item["item"] for item in zigbang_dict_list ]
len(items)

60

In [38]:
items

[{'_floor': '고층/15층',
  'address1': '서울시 마포구 성산동',
  'address2': '593-3',
  'address3': None,
  'agent_address1': '서울특별시 마포구 모래내로1길 8 나동？다동 103호 (성산동)',
  'agent_comment': '\n━━━━━━━━━━━━━━━━━━━━━━━\n\n월드컵삼영공인중개사사무소 ☎02-309-3030\n\n친절하고 신속.정확한 중개 약속드립니다!\n많은 물건이 있으니 편하게 전화주세요~ \n\n오늘도 역시 밝게 웃을 수 있는 하루되시길 바랍니다^^ ',
  'agent_email': '3030land@naver.com',
  'agent_lat': 37.56481051112199,
  'agent_lng': 126.904238840368,
  'agent_local1': '서울시',
  'agent_local2': '마포구',
  'agent_mobile': '0507-1282-2194',
  'agent_name': '월드컵삼영공인중개사(서귀선)',
  'agent_no': 0,
  'agent_phone': '02-309-3030',
  'bjd_code': '1144012500',
  'bonbun_code': '593',
  'bubun_code': '3',
  'building': {'address': '서울시 마포구 성산동 593-3',
   'address2': '593-3',
   'building_id': 20073,
   'count': 107,
   'elevator': '1',
   'established': '2018.06',
   'floor': '15층',
   'lat': 37.56548690795898,
   'lng': 126.90431213378906,
   'local1': '서울시',
   'local2': '마포구',
   'local3': '성산동',
   'name': '동우자인채스토리상암',
   'rooms'

In [52]:
zigbang = client.crawling.zigbang
zigbang

Collection(Database(MongoClient(host=['13.124.198.120:27017'], document_class=dict, tz_aware=False, connect=True), 'crawling'), 'zigbang')

In [53]:
zigbang.insert_many(items)

In [54]:
# 렌트 비용이 50만원 이하의 데이터를 추출
QUERY = {"rent":{"$lte":50}}
documents = zigbang.find(QUERY)
datas = list(documents)
len(datas)

44

In [56]:
# 결과를 판다스의 데이터 프레임으로 만들기
df = pd.DataFrame(datas) 
df.tail()

,_floor,_id,address1,address2,address3,agent_address1,agent_comment,agent_email,agent_lat,agent_lng,...,updated_at2,user_email,user_has_no_penalty,user_has_penalty,user_intro,user_mobile,user_name,user_no,user_phone,view_count
39,4층,5b5934eb46fb2a4d5313a0c6,서울시 마포구 망원동,435-45,None,서울특별시 마포구 서교동 336-15 102호,◈ 공인중개사 자격 있는 미모의 김다운실장입니다.\n◈ 김실장의 흰모닝으로 여러분의...,narayong83@naver.com,37.554131,126.927803,...,2018-07-23,narayong83@naver.com,True,False,,0507-1280-4675,대표공인중개사 김다운,2609060,0507-1280-4675,60
40,1층,5b5934eb46fb2a4d5313a0c8,서울시 마포구 합정동,445-10,None,서울특별시 마포구 합정동 442-2,"저희가 100%직접 촬영한 사진입니다. \n실매물, 실가격으로 제공해드리고 있으며 ...",green7003@naver.com,37.551884,126.905045,...,2018-07-25,green7003@naver.com,True,False,믿고 책임질 수 있는 매물만 중개하는 초록공인중개사무소입니다.\n믿을 수 있는 매물...,0507-1282-2016,대표공인중개사 신지연,829956,0507-1282-2016,544
41,3층,5b5934eb46fb2a4d5313a0ca,서울시 마포구 망원동,404-15,None,서울시 마포구 합정동 383-9,,a941010@naver.com,37.549435,126.912498,...,2018-07-24,iys0124@naver.com,True,False,안녕하세요. 직방회원여러분\nA++ 매물을 최다보유하고 있는 맞춤서비스 마포대표부동...,0507-1282-0029,중개보조원 장용석,700213,0507-1282-0029,217
42,4층,5b5934eb46fb2a4d5313a0cc,서울시 마포구 망원동,455-66,None,"서울특별시 마포구 독막로 61 , 제2층","★ 드림공인중개사 사무소\n\n* 마포구 원룸/빌라, 상가점포, 건물 전문으로 다양...",dream_realty@naver.com,37.548006,126.920393,...,2018-07-26,dream_realty@naver.com,True,False,,0507-1281-5789,소속공인중개사 원유호,4149817,0507-1281-5789,540
43,2층,5b5934eb46fb2a4d5313a0cd,서울시 마포구 망원동,468-25,None,"서울특별시 마포구 성미산로 21 101호 (성산동, 진흥 하임시티)",,phj0280@hanmail.net,37.558674,126.910443,...,2018-07-12,phj0280@hanmail.net,True,False,,0507-1282-1766,대표공인중개사 박현정,894577,0507-1282-1766,123


In [58]:
df[["deposit","rent","size"]].tail()

,deposit,rent,size
39,25000,0,14.0
40,1000,50,12.0
41,3000,25,5.0
42,20000,0,13.5
43,23500,0,13.0


In [48]:
# drop - database
client.drop_database("crawling")

In [59]:
# drop - collection
client.crawling.drop_collection("zigbang")

{'nIndexesWas': 1, 'ns': 'crawling.zigbang', 'ok': 1.0}

##### Quiz. deposit이 3000이하의 데이터를 "agent_mobile", "deposit", "rent" 컬럼이 출력되도록 df 만드세요.

In [63]:
zigbang = client.crawling.zigbang
documents = zigbang.find( {"deposit":{"$lte":3000}}, \
{"_id":False, "agent_mobile":True, "deposit":True, "rent":True} )
documents

In [64]:
datas = list(documents)
len(datas)

31

In [65]:
df = pd.DataFrame(datas)
df.tail()

,agent_mobile,deposit,rent
26,0507-1281-3804,2000,40
27,0507-1280-8642,500,60
28,0507-1280-3619,500,55
29,0507-1281-1007,1000,80
30,0507-1281-5289,2000,40
